# A/B Testing
* update: 2021/12/24

## Preparation

In [33]:
# import packages
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
# define locations
input_path = '../input/'

# define file name
customer_demographics_file_name = 'customer_data.csv'
customer_subcriptions_file_name = 'inapp_purchases.csv'

# define files
customer_demographics_file = input_path + customer_demographics_file_name
customer_subcriptions_file = input_path + customer_subcriptions_file_name

In [3]:
# read in data
df_customer_demographics = pd.read_csv(customer_demographics_file)
df_customer_subcriptions = pd.read_csv(customer_subcriptions_file)

In [4]:
df_customer_demographics.shape

(10000, 6)

In [5]:
df_customer_demographics.head()

,uid,reg_date,device,gender,country,age
0,54030035,2017-06-29,and,M,USA,19
1,72574201,2018-03-05,iOS,F,TUR,22
2,64187558,2016-02-07,iOS,M,USA,16
3,92513925,2017-05-25,and,M,BRA,41
4,99231338,2017-03-26,iOS,M,FRA,59


In [6]:
# rename reg_date to date
df_customer_demographics = df_customer_demographics.rename( columns={ 'reg_date' : 'date' } )

In [7]:
df_customer_demographics.head()

,uid,date,device,gender,country,age
0,54030035,2017-06-29,and,M,USA,19
1,72574201,2018-03-05,iOS,F,TUR,22
2,64187558,2016-02-07,iOS,M,USA,16
3,92513925,2017-05-25,and,M,BRA,41
4,99231338,2017-03-26,iOS,M,FRA,59


In [8]:
df_customer_subcriptions.shape

(9006, 4)

In [9]:
df_customer_subcriptions.head()

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


## EDA

In [10]:
# Merge on the 'uid' field
df_uid_combined_data = df_customer_subcriptions.merge(df_customer_demographics, on=['uid'], how='inner')
df_uid_combined_data.shape

(9006, 9)

In [11]:
df_uid_combined_data.head()

,date_x,uid,sku,price,date_y,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17


In [12]:
# Merge on the 'uid' and 'date' field
df_uid_date_combined_data = df_customer_subcriptions.merge(df_customer_demographics, on=['uid', 'date'], how='inner')
df_uid_date_combined_data.shape

(35, 8)

In [13]:
df_uid_date_combined_data.head()

,date,uid,sku,price,device,gender,country,age
0,2016-03-30,94055095,sku_four_599,599,iOS,F,BRA,16
1,2015-10-28,69627745,sku_one_199,199,and,F,BRA,18
2,2017-02-02,11604973,sku_seven_1499,499,and,F,USA,16
3,2016-06-05,22495315,sku_four_599,599,and,F,USA,19
4,2018-02-17,51365662,sku_two_299,299,iOS,M,TUR,16


## KPI

In [14]:
# Calculate the mean purchase price 
purchase_price_mean = df_customer_subcriptions.price.agg('mean')
purchase_price_mean

406.77259604707973

In [15]:
# Calculate the mean and median purchase price 
purchase_price_summary = df_customer_subcriptions.price.agg(['mean', 'median'])
purchase_price_summary

mean      406.772596
median    299.000000
Name: price, dtype: float64

In [16]:
# Calculate the mean and median of price and age
purchase_summary = df_uid_combined_data.agg({'price': ['mean', 'median'], 'age': ['mean', 'median']})
purchase_summary

,price,age
mean,406.772596,23.922274
median,299.000000,21.000000


### Grouping & aggregating

In [17]:
# Group the data 
grouped_purchase_data = df_uid_combined_data.groupby(by = ['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504    299  179.984378
       M       416.237308    499  195.001520
iOS    F       404.435330    299  181.524952
       M       405.272401    299  196.843197


In [18]:
# define current_date
current_date = pd.to_datetime('2018-03-17')
current_date

Timestamp('2018-03-17 00:00:00')

In [19]:
# Compute max_purchase_date 
# Subtract timedelta(days=28) from current_date to find the last date that we will count purchases from. The current_date variable has already been defined.
max_purchase_date = current_date - timedelta(days=28)
max_purchase_date

Timestamp('2018-02-17 00:00:00')

In [24]:
# make a copy of previous dataframe and rename columns
df_purchase_data = df_uid_combined_data
df_purchase_data['date'] = df_purchase_data['date_x'].astype('datetime64[ns]')
df_purchase_data['reg_date'] = df_purchase_data['date_y'].astype('datetime64[ns]')
df_purchase_data.head()

,date_x,uid,sku,price,date_y,device,gender,country,age,date,reg_date
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17,2017-07-10,2017-06-26
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17,2017-07-15,2017-06-26
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17,2017-11-12,2017-06-26
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17,2017-09-26,2017-01-05
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17,2017-12-01,2017-01-05


In [25]:
df_purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9006 entries, 0 to 9005
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date_x    9006 non-null   object        
 1   uid       9006 non-null   int64         
 2   sku       9006 non-null   object        
 3   price     9006 non-null   int64         
 4   date_y    9006 non-null   object        
 5   device    9006 non-null   object        
 6   gender    9006 non-null   object        
 7   country   9006 non-null   object        
 8   age       9006 non-null   int64         
 9   date      9006 non-null   datetime64[ns]
 10  reg_date  9006 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(6)
memory usage: 1.1+ MB


In [27]:
df_purchase_data = df_purchase_data[['date', 'uid', 'sku', 'price', 'reg_date', 'device', 'gender', 'country', 'age']]
df_purchase_data.head()

,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17


In [29]:
# Filter to only include users who registered before our max date
df_purchase_data_filt = df_purchase_data[df_purchase_data.reg_date < max_purchase_date]

In [30]:
# Filter to contain only purchases within the first 28 days of registration
df_purchase_data_filt = df_purchase_data_filt[(df_purchase_data_filt.date <=
                                         df_purchase_data_filt.reg_date + 
                                         timedelta(days=28))]

In [31]:
# Output the mean price paid per purchase
print(df_purchase_data_filt.price.mean())

414.4237288135593


### Average purchase price by cohort
Building on the previous exercise, let's look at the same KPI, average purchase price, and a similar one, median purchase price, within the first 28 days. Additionally, let's look at these metrics not limited to 28 days to compare.

We can calculate these metrics across a set of cohorts and see what differences emerge. This is a useful task as it can help us understand how behaviors vary across cohorts.

Note that in our data the price variable is given in cents.

### Instructions 1/3
Use np.where to create an array month1 containing:

* the price of the purchase purchase, if
   1. the user registration .reg_date occurred at most 28 days ago (i.e. before max_reg_date), and
   2. the date of purchase .date occurred within 28 days of registration date .reg_date;
* NaN, otherwise.

In [34]:
# Set the max registration date to be one month before today
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values:
month1 = np.where((df_purchase_data.reg_date < max_reg_date) &
                    (df_purchase_data.date < df_purchase_data.reg_date + timedelta(days=28)),
                  df_purchase_data.price, 
                  np.NaN)
                 
# Update the value in the DataFrame 
df_purchase_data['month1'] = month1

### Instructions 2/3
Now, group purchase_data by gender and then device using the .groupby() method.

In [35]:
# Group the data by gender and device 
purchase_data_grp = df_purchase_data.groupby(by=['gender', 'device'], as_index=False)

### Instructions 3/3
Aggregate the "mean" and "median" of both 'month1' and'price' using the .agg() method in the listed order of aggregations and fields.

In [36]:
# Aggregate the month1 and price data 
purchase_summary = purchase_data_grp.agg(
                        {'month1': ['mean', 'median'],
                        'price': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  388.204545  299.0  400.747504    299
1      F    iOS  432.587786  499.0  404.435330    299
2      M    and  413.705882  399.0  416.237308    499
3      M    iOS  433.313725  499.0  405.272401    299
